In [80]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv(r"/Users/ziadali/Documents/Personal Projects/Projects/Customer Churn Prediction/processed_data.csv")

In [14]:
df.head(1)

,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value
0,Male,Young,0,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1


In [ ]:
#drop useless features
df.drop(columns=['CustomerID','Zip Code','Churn Score','CLTV','Monthly_Charges_bin','Total_Charges_bin','Tenure_Months_bin','City'] , inplace= True)

In [ ]:
#convert binary_objects features to integers
Gender_dic = {'Male' : 1 , 'Female' : 0}
df['Gender'] = df['Gender'].replace(Gender_dic)
Senior_citizen_dic = {'Young' : 0 , 'Senior' : 1}
df['Senior Citizen'] = df['Senior Citizen'].replace(Senior_citizen_dic)
Dependents_dic = {'Yes' : 1 , 'No' : 0}
df['Dependents'] = df['Dependents'].replace(Dependents_dic)
Phone_service_dic = {'Yes' : 1 , 'No' : 0}
df['Phone Service'] = df['Phone Service'].replace(Phone_service_dic)
Paperless_Billing_dic = {'Yes' : 1 , 'No' : 0}
df['Paperless Billing'] = df['Paperless Billing'].replace(Paperless_Billing_dic)

/var/folders/fc/6b57d43d15d8m9h3x_m362jh0000gn/T/ipykernel_5111/553212940.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Paperless Billing'] = df['Paperless Billing'].replace(Paperless_Billing_dic)


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             7043 non-null   int64  
 1   Senior Citizen     7043 non-null   int64  
 2   Partner            7043 non-null   int64  
 3   Dependents         7043 non-null   int64  
 4   Tenure Months      7043 non-null   int64  
 5   Phone Service      7043 non-null   int64  
 6   Multiple Lines     7043 non-null   object 
 7   Internet Service   7043 non-null   object 
 8   Online Security    7043 non-null   object 
 9   Online Backup      7043 non-null   object 
 10  Device Protection  7043 non-null   object 
 11  Tech Support       7043 non-null   object 
 12  Streaming TV       7043 non-null   object 
 13  Streaming Movies   7043 non-null   object 
 14  Contract           7043 non-null   object 
 15  Paperless Billing  7043 non-null   int64  
 16  Payment Method     7043 

In [67]:
#spliting the data
y = df['Churn Value']
X = df.drop(columns=['Churn Value'])

In [68]:
numerical_features = [
    'Tenure Months',
    'Monthly Charges',
    'Total Charges'
]
binary_features = [
    'Gender',
    'Phone Service',
    'Senior Citizen',
    'Dependents',
    'Partner',
    'Paperless Billing'
]
categorical_features = [
    'Multiple Lines',
    'Internet Service',
    'Online Security',
    'Online Backup',
    'Device Protection',
    'Tech Support',
    'Streaming TV',
    'Streaming Movies',
    'Contract',
    'Payment Method'
]

In [69]:
#num_column_transform
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [70]:
#cat_column_transform
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(
        drop='first',
        handle_unknown='ignore'
    ))
])

In [71]:
#columns_trans
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numerical_features),
        ('cat', cat_transformer, categorical_features)
    ],
    remainder='passthrough'
)

In [72]:
model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000
)

In [73]:
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', model)
])

In [74]:
#train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , stratify=y ,random_state=42)

In [75]:
#train_model
pipeline.fit(X_train, y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [77]:
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

In [81]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.73      0.81      1035
           1       0.51      0.78      0.62       374

    accuracy                           0.74      1409
   macro avg       0.71      0.76      0.71      1409
weighted avg       0.80      0.74      0.76      1409

